<h1>
Neural Network Attempt
<h1>

<h2>Import Necessary Libraries<h2>

In [ ]:
import numpy as np
import pandas as pd
import math

<h2> Load Data <h2>
<h3>
<p>Output: Training <p>
<p>Correct: Target <p>
<h3>

In [48]:
outputfn = "OutputTiny.csv"
correctfn = "CorrectTiny.csv"
outputdf = pd.read_csv(outputfn)
correctdf = pd.read_csv(correctfn)

<h2> Calibrate the Neural Network <h1>

In [219]:
lr = 0.00000007 # learning rate
#worked best experimentally likely due to the large volume of data
hn = 4 #hidden nodes
inputs = 10 #input and output nodes
n = 2 * inputs * hn # total weights
weights = np.random.uniform(0, 1, n) + 1.0j * np.random.uniform(0, 1, n)
biases = [0]*hn #for clarity in formulas, not used

<h2> Load Helper Functions / NN Structure <h2>

In [220]:
# lines marked R are to be used when using complex ReLu
def complex_relu(num):
    return complex(max(num.real,0) + 1.0j*max(num.imag,0))
def dev_complex_relu(num):
    imaginary = 0
    real_num = 0
    if num.imag > 0:
        imaginary = 1
    if num.real > 0:
        real_num = 1
    return complex(real_num + 1.0j*imaginary)
def complex_sigmoid(num):
    return complex(1/(1 + np.exp(-1*num.real)) + (1/(1 + np.exp(-1*num.imag)))*1.0j)
def forward_propagation(inputA):
    hidden_layer = biases
    for k in range(0,len(hidden_layer)):
        for i in range(0,len(inputA)):
            hidden_layer[k] +=  complex(inputA[i].replace('i','j'))*weights[i+(len(inputA)*k)]
        #hidden_layer[k] = complex_relu(hidden_layer[k]) #R
        hidden_layer[k] = complex_sigmoid(hidden_layer[k])
    output = [0]*len(inputA)
    for k in range(0,len(hidden_layer)):
        for i in range(0,len(inputA)):
            output[i] +=  hidden_layer[k]*(weights[i+(len(hidden_layer)*len(inputA) + (len(inputA)*k))])
    for i in range(0, len(output)):
        output[i] = complex_sigmoid(output[i])
    return output

def back_propagation(start, end):
    bp_gradient = [0]*len(weights)
    bp_gradient_b = [0]*len(biases)
    for x in range(start, end+1):
        outputdf_list =outputdf[str(x)].to_list() 
        correctdf_list =correctdf[str(x)].to_list()  
        output = forward_propagation(outputdf_list)
        for i in range(0,len(bp_gradient)):
            h = math.floor(i/inputs)
            if i <= n/2:
                sum = 0
                mult = 2*(output[i%inputs]- complex(correctdf_list[i%inputs].replace('i','j')))
                for j in range(0, len(output)):
                    #sum += weights[400+j] * dev_complex_relu(output[j]) * mult #R
                    sum += weights[int(n/2)+j] * output[j] * mult
                temp = complex_sigmoid(weights[i]*complex(outputdf_list[i%inputs].replace('i','j'))+biases[min(h,hn-1)])           
                bp_gradient[i] += complex(outputdf_list[i%inputs].replace('i','j')) * temp*(1-temp) * sum    
                #bp_gradient[i] += (complex(outputdf_list[i%inputs].replace('i','j')) * dev_complex_relu(weights[i] #R
                # * complex(outputdf_list[i%inputs].replace('i','j'))+biases[min(h,hn)]) * sum) #R

            else:
                sum = 0
                for j in range(0, len(outputdf_list)):
                    sum += complex(outputdf_list[j].replace('i','j'))*weights[j+(inputs*h)]
                temp = complex_sigmoid(sum)
                bp_gradient[i] += sum * temp * (1-temp) * 2 *(output[i%inputs]- complex(correctdf_list[i%inputs].replace('i','j')))
                #bp_gradient[i] += sum * dev_complex_relu(sum) * 2 *(output[i%100]- complex(correctdf_list[i%100].replace('i','j'))) #R
    for x in range(start, end+1):
        outputdf_list =outputdf[str(x)].to_list() 
        correctdf_list =correctdf[str(x)].to_list()  
        output = forward_propagation(outputdf_list)
    for i in range(0, len(weights)):
        weights[i] = weights[i] - lr*bp_gradient[i]/(start-end)


        

<h2> Train the NN <h2>

In [ ]:
#Assuming 100000 data points with batch size = 20 for 5000 batches
for i in range(1,5001):
    back_propagation((i-1)*20+1, i*20)

<h1> Test the Error Rates <h1>

In [ ]:
error_nn_r, error_nn_i, error_c_r, error_c_i = 0
for i in range(1, len(outputdf)):
    out = forward_propagation(outputdf[str(i)].to_list())
    inp = outputdf[str(i)].to_list() 
    cor = correctdf[str(i)].to_list() 
    div = len(inp)
    for j in range(0, len(inp)):
        error_nn_r += (out[j].real - complex(cor[j].replace('i','j')).real)**2
        error_nn_i += (out[j].imag - complex(cor[j].replace('i','j')).imag)**2
        error_c_r  += (complex(inp[j].replace('i','j')).real - complex(cor[j].replace('i','j')).real)**2
        error_c_i  += (complex(inp[j].replace('i','j')).imag - complex(cor[j].replace('i','j')).imag)**2
print(f"Neural Network Error (Real): {error_nn_r/(div*len(outputdf))}")
print(f"Neural Network Error (Imag): {error_nn_i/(div*len(outputdf))}")
print(f"Transfer Estim Error (Real): {error_c_r/(div*len(outputdf))}")
print(f"Transfer Estim Error (Real): {error_c_i/(div*len(outputdf))}")


<h2>Save Weights and Biases<h2>

In [7]:

w_file = open('weights.txt','w')
b_file = open('biases.txt', 'w') 
w_file.writelines([str(x)+"\n" for x in weights])
b_file.writelines([str(x)+"\n" for x in biases])